In [2]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
timestamp = pd.read_csv('timestamp.tsv', header=None, sep='\t')
flow = pd.read_csv('flow.tsv', header=None, sep='\t')
occupancy = pd.read_csv('occupancy.tsv', header=None, sep='\t')
speed = pd.read_csv('speed.tsv', header=None, sep='\t')

In [5]:
fpd = pd.concat([flow[0], flow[1], flow[2]]).reset_index()[0]
opd = pd.concat([occupancy[0], occupancy[1], occupancy[2]]).reset_index()[0]
spd = pd.concat([speed[0], speed[1], speed[2]]).reset_index()[0]

In [6]:
mvpd = pd.concat([fpd,spd, opd], axis=1)
mvpd.columns = ['flow', 'speed', 'occupancy']

In [17]:
mvpd = mvpd.dropna(how='any')

In [34]:
len(mvpd)

11326053

In [8]:
mv = mvpd.as_matrix(columns=None)
mv

array([[  8.,  68.,   3.],
       [  9.,  67.,   4.],
       [ 11.,  66.,   4.],
       ..., 
       [  3.,   0.,   0.],
       [  3.,   0.,   0.],
       [  5.,   0.,   0.]])

In [10]:
mvpd.to_csv('1160.txt', sep='\t', encoding='utf-8',index=False)

In [86]:
mvpd.head()

,flow,speed,occupancy
0,8,68.0,3.0
1,9,67.0,4.0
2,11,66.0,4.0
3,-8,71.0,3.0
4,5,65.0,2.0


In [11]:
pdf = mvpd[((mvpd.flow > 0) & (mvpd.speed > 0) & (mvpd.occupancy >= 0) & (mvpd.occupancy <=100))
                   | ((mvpd.flow == 0) & (mvpd.speed == 0) & (mvpd.occupancy == 0))
                   | ((mvpd.flow == 0) & (mvpd.speed == 0) & (mvpd.occupancy ==100))]
ndf = mvpd[~(((mvpd.flow > 0) & (mvpd.speed > 0) & (mvpd.occupancy >= 0) & (mvpd.occupancy <=100))
                      | ((mvpd.flow == 0) & (mvpd.speed == 0) & (mvpd.occupancy == 0))
                      | ((mvpd.flow == 0) & (mvpd.speed == 0) & (mvpd.occupancy ==100)))]


In [12]:
ndf["probability"] =0
ndf.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,flow,speed,occupancy,probability
3,-8,71.0,3.0,0
7,0,65.0,0.0,0
35,-8,69.0,4.0,0
50,0,68.0,0.0,0
52,0,62.0,0.0,0


In [13]:
mean1 = pdf['flow'].mean()
mean2 = pdf['speed'].mean()
mean3 = pdf['occupancy'].mean()

In [14]:
def computex2(row):
   if ((row['flow']<0) | (row['speed']<0) | (row['occupancy']<0)):
       x2 = float("inf")
   else:
       x2 = ((row['flow'] - mean1)**2/mean1) + ((row[1] - mean2)**2/mean2) + ((row[2] - mean3)**2/mean3) # Simple calc for example
   return x2

In [16]:
pdf['chi2'] = pdf.apply(computex2, axis=1)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
ws_range = np.arange(0, 27780 + 1, 10)

In [119]:
x = pd.cut(pdf['chi2'],ws_range)

In [21]:
a = pdf.groupby(pd.cut(pdf['chi2'],ws_range))
pdf['probability']=a['chi2'].transform('count')/len(pdf['chi2'])

In [24]:
pdf.head()

,flow,speed,occupancy,chi2,probability
0,8,68.0,3.0,58.054900,0.073428
1,9,67.0,4.0,56.753669,0.073428
2,11,66.0,4.0,56.766960,0.073428
4,5,65.0,2.0,49.301779,0.068037
5,7,66.0,2.0,52.045195,0.073428


In [25]:
sortdf = pdf.sort(columns='probability', axis=0, ascending=True, na_position='last')

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [32]:
pdf = sortdf[['flow','speed','occupancy','probability']]
pdf.head()

,flow,speed,occupancy,probability
8137265,12,1.0,75.0,1.170225e-07
7770836,238,41.0,4.0,1.170225e-07
7710423,243,3.0,4.0,1.170225e-07
7584348,227,5.0,4.0,1.170225e-07
7776953,267,8.0,4.0,1.170225e-07


In [35]:
resultdf = pd.concat([ndf, pdf]).reset_index().drop('index', 1)

In [37]:
sample = resultdf[::100]
sample[['flow', 'speed', 'occupancy']] = sample[['flow', 'speed', 'occupancy']].astype(int)

In [42]:
sample.head()

,flow,speed,occupancy,probability
0,-8,71,3,0.0
100,0,72,0,0.0
200,0,65,0,0.0
300,0,72,0,0.0
400,0,72,0,0.0


In [43]:
sample.to_csv('1160.txt', sep='\t', encoding='utf-8',index=False)